## Solving metabolic stoichiometry and energetics

Biomass comes from: CH<sub>1.8</sub>O<sub>0.5</sub>N<sub>0.2</sub>

In [1]:
import sympyprinting
import sympy
import pandas as pd

In [2]:
thermo = pd.read_excel('Reactant_Thermodynamics.xlsx', convert_float=False)
thermo.set_index(thermo.symbols.values, inplace=True)

In [3]:
import sympy
import numpy as np

In [4]:
#NH4, H2O, NO2, H, e, O2, C6H12O6, C2H3O2, HCO3, H2, C2O4, CH4, C2H5OH, C2H3O2  = sympy.symbols(["NH4","H2O","NO2","H","e","O2", "C6H12O6","C2H3O2","HCO3","H2", "C2O4", "CH4", "C2H5OH","C2H3O2"])

#From the dataframe - build up all the symbols
for s in thermo.symbols.values:
    exec("%s = sympy.symbols('%s')" %(s,s))
#exec("%s = sympy.symbols(thermo.symbols.values)" % ','.join(thermo.symbols.values))

In [5]:
from collections import OrderedDict
## This only works if the od "donors" are supplied in order of donation... so because NO3 as an N donor contains both N and O, 
# it must be added in N,O,H order. Also H2O for O before H ...
def BalanceStoichiometry(eq, od):
    cureq = eq
    for k,v in od.items():
        #print k,v
        nXs = cureq.subs(thermo.loc[map(str,(cureq.free_symbols))][["symbols",k]].values)
        #print cureq
        cureq = cureq - nXs * v
    return cureq
        
        

In [6]:
import chempy
from chempy import reactionsystem
from chempy import balance_stoichiometry

In [7]:

subsdict = {x: chempy.Substance.from_formula(x) for x in ['C2H5OH','HCO3-','NH4+','H+', 'H2O']}
X=chempy.Substance(name='X', charge=0, composition={1: 1.8, 6: 1, 7: 0.2, 8: 0.5})
subsdict["X"] = X

balance_stoichiometry({'C2H5OH','HCO3-','NH4+','H+'},{ 'H2O', 'C10H18N2O5'})

({'C2H5OH': 7, 'H+': 2, 'HCO3-': 6, 'NH4+': 4}, {'C10H18N2O5': 2, 'H2O': 15})

# FROM HERE!!! :

In [8]:
R = 0.00831
T = 298.14
from numpy import log 


def reacToEquation(balanced_reaction, debug=False):
    b = balanced_reaction
    if debug:
        print (b)
        
    thermolookup = thermo.set_index(thermo.structure.values)
    thermolookup.rename(index={"CH1.8O0.5N0.2":"C10H18O5N2"}, inplace=True)
    to_join = ["-%d*%s" %(i,thermolookup.loc[k]["symbols"]) for k,i in b[0].items()] + ["%d*%s" %(i,thermolookup.loc[k]["symbols"]) for k,i in b[1].items()]
    if debug:
        print ("To Join:")
        print (to_join)
    exec('reac = ' + ' + '.join(to_join))
    return reac

def calcDG(equ, debug=False):
    coeffs_dict = sympy.N(equ).as_coefficients_dict()

        
    c = thermo.loc[[t.name for t in coeffs_dict.keys()]]["Gf0"]
    v = coeffs_dict.values()
    if debug:
        print (c)
    return (c * v).sum()

#catab_A = reacToEquation(cat_A, debug=True)

In [9]:
def calcGDis(csource):
    from numpy import exp
    
    gamma = thermo["g"].loc[csource]
    NoC = thermo["C"].loc[csource]
    return 200 + 18 * (6 - NoC)**1.8 + exp(((-0.2 - gamma)**2)**0.16 * (3.6+0.4 * NoC))

In [10]:
calcGDis("c_CO2")

1088.0937132378845

In [11]:
def getLaTeXReacts(anab,catab):
    
    for m in [("Catab", catab), ("Anab", anab)]:
        reacts = []
        prods = [] 
        metab = m[1]
        for r,c in metab.as_coefficients_dict().items():
            if c < 0:
                reacts.append('%0.1f %s' % (c*-1.0,chempy.Substance.from_formula(thermo.loc[r.name].structure).latex_name))
            else:
                #Special case for c_X
                if r == c_X:
                    prods.append('%0.2f CH_{1.8}O_{0.5}N_{0.2}' % c)
                else:
                    prods.append('%0.2f.%s' % (c,chempy.Substance.from_formula(thermo.loc[r.name].structure).latex_name))
        rstr = '$' + '+'.join(reacts) + ' \\rightarrow '  + '+'.join(prods) + '$'
        print ("%s\n\n%s\n\n" %(m[0],rstr))
        
        
def getHTMLReacts(anab,catab):
    
    for m in [("Catab", catab), ("Anab", anab)]:
        reacts = []
        prods = [] 
        metab = m[1]
        for r,c in metab.as_coefficients_dict().items():
            if c < 0:
                reacts.append('%0.1f %s' % (c*-1.0,chempy.Substance.from_formula(thermo.loc[r.name].structure).html_name))
            else:
                #Special case for c_X
                if r == c_X:
                    prods.append('%0.1f CH<sub>1.8</sub>O<sub>0.5</sub>N<sub>0.2</sub>' % c)
                else:
                    prods.append('%0.1f.%s' % (c,chempy.Substance.from_formula(thermo.loc[r.name].structure).html_name))
        rstr = '+'.join(reacts) + ' &rarr; '  + '+'.join(prods) 
        print ("#### %s\n%s" %(m[0],rstr))

In [12]:
thermo

,name,structure,phase,Gf0,Hf0,g,COD,Y,Gy,charge,C,H,N,O,S,symbols,electrons
c_e_n1,Electron,e-1,aq,0.00,0.00,0.0,0.00,0.0,NaN,-1.0,0.0,0.0,0.0,0.0,0.0,c_e_n1,1.0
c_H_p1,Proton,H+1,aq,0.00,0.00,1.0,0.00,0.0,NaN,1.0,0.0,1.0,0.0,0.0,0.0,c_H_p1,0.0
c_H2,Hydrogen,H2,g,0.00,0.00,0.0,2.00,-40.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,c_H2,2.0
c_O2,Oxygen,O2,g,0.00,0.00,0.0,0.00,0.0,NaN,0.0,0.0,0.0,0.0,2.0,0.0,c_O2,12.0
c_H2O,Water,H2O,l,-237.20,-285.80,-2.0,4.00,78.6,NaN,0.0,0.0,2.0,0.0,1.0,0.0,c_H2O,8.0
c_N2,Dinitrogen,N2,g,0.00,0.00,0.0,10.00,72.1,NaN,0.0,0.0,0.0,2.0,0.0,0.0,c_N2,10.0
c_NO,Nitric oxide,NO,g,86.60,90.20,2.0,3.00,38.9,NaN,0.0,0.0,0.0,1.0,1.0,0.0,c_NO,11.0
c_N2O,Dinotrogenoxide,N2O,g,104.20,82.00,1.0,8.00,57.5,NaN,0.0,0.0,0.0,2.0,1.0,0.0,c_N2O,16.0
c_NH2OH,Hydroxylamine,NH2OH,g,-43.60,NaN,-1.0,6.00,21.2,NaN,0.0,0.0,1.0,1.0,1.0,0.0,c_NH2OH,12.0
c_N2H4,Hydrazine,N2H4,g,159.17,95.30,-2.0,14.00,28.7,NaN,0.0,0.0,4.0,2.0,0.0,0.0,c_N2H4,14.0


## Glucose aerobic respiration

$ C_6H_{12}O_6 + 6O_2 \rightarrow 6CO_2 + 6H_2O$ 


e<sup>-</sup> donor:

$ C_6H_{12}O_6 + H_2O \rightarrow CO_2 + H^+ + e^- $

e<sup>-</sup> acceptor:

$ O_2 + H^+ \rightarrow H_2O $

Anabolism:


In [13]:
catab_donor = c_C6H12O6

cat_A = balance_stoichiometry({"O2","e-1","H+1"}, {"H2O"})
cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"HCO3-1", "e-1", "H+1"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C6H12O6','NH4+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
DGcatab_cor = calcDG(catab)
DGanab_cor = calcDG(anab)

#Corrections
DGcatab_cor += R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor += R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

DGdis = calcGDis("c_C6H12O6")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print ("\n\ncatab = %f" % DGcatab_cor)
print ("anab = %f" % DGanab_cor)
print ("dis = %f" % DGdis)
print ("Yield = %f" % (1.0/lambdacat))
print ("\n\n\n#### Yield = %0.4f" %  (1.0/lambdacat) )
getHTMLReacts(anab,catab)

Catabolism:
-c_C6H12O6 + 6.0*c_HCO3_n1 + 6.0*c_H_p1 - 6.0*c_O2

Anabolism:
-0.166666666666667*c_C6H12O6 + 0.5*c_H2O - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1


catab = -2841.199689
anab = -24.363320
dis = 236.050884
Yield = 13.421666



#### Yield = 13.4217
#### Catab
6.0 O<sub>2</sub>+1.0 C<sub>6</sub>H<sub>12</sub>O<sub>6</sub> &rarr; 6.0.HCO<sub>3</sub><sup>-</sup>+6.0.H<sup>+</sup>
#### Anab
0.2 C<sub>6</sub>H<sub>12</sub>O<sub>6</sub>+0.2 NH<sub>4</sub><sup>+</sup> &rarr; 1.0 CH<sub>1.8</sub>O<sub>0.5</sub>N<sub>0.2</sub>+0.4.H<sub>2</sub>O+0.2.H<sup>+</sup>+0.1.HCO<sub>3</sub><sup>-</sup>


## Anaerobic glucose fermentation

$ C_6H_{12}O_6 + H_2O \rightarrow C_2H_5O^- + HCO_3 + H_2 $

In [197]:
catab_donor = c_C6H12O6
catab_acceptor = c_C6H12O6

cat_A = balance_stoichiometry({"e-1","H+1"}, {"H2"})
#cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"HCO3-1", "C2H3O2-1", "e-1", "H+1"})
cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"C3H4O5-2", "e-1", "H+1"})

D_r = cat_D[0]
D_p = cat_D[1]
#Re-insert the correct ratio of products
D_p["HCO3-1"] = D_p["C3H4O5-2"]
D_p["C2H3O2-1"] = D_p["C3H4O5-2"]
#Clear out the "fake" reactant
del D_p["C3H4O5-2"]

cat_D=(D_r,D_p)

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C6H12O6','NH4+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D

print anab
DGcatab_cor = calcDG(catab)
DGanab_cor = calcDG(anab)

#Corrections
DGcatab_cor += R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor += R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
DGdis = calcGDis("c_C6H12O6")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)
print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)


Catabolism:
2.0*c_C2H3O2_n1 - c_C6H12O6 + 4.0*c_H2 - 4.0*c_H2O + 2.0*c_HCO3_n1 + 4.0*c_H_p1

Anabolism:
-0.166666666666667*c_C6H12O6 + 0.5*c_H2O - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1
0.05*c_C2H3O2_n1 - 0.191666666666667*c_C6H12O6 + 0.4*c_H2O + 0.05*c_HCO3_n1 + 0.3*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -203.733126
anab = -29.499984
dis = 236.050884
Yield = 0.986358



### Yield = 0.9864
Catab

$4.0 H_{2}O+1.0 C_{6}H_{12}O_{6} \rightarrow 2.0.C_{2}H_{3}O_{2}^{-}+4.0.H^{+}+2.0.HCO_{3}^{-}+4.0.H_{2}$


Anab

$0.2 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 0.1.C_{2}H_{3}O_{2}^{-}+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.4.H_{2}O+0.3.H^{+}+0.1.HCO_{3}^{-}$




### Yield = 0.9864
Catab

$4.0 H_{2}O+1.0 C_{6}H_{12}O_{6} \rightarrow 2.0.C_{2}H_{3}O_{2}^{-}+4.0.H^{+}+2.0.HCO_{3}^{-}+4.0.H_{2}$


Anab

$0.2 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 0.1.C_{2}H_{3}O_{2}^{-}+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.4.H_{2}O+0.3.H^{+}+0.1.HCO_{3}^{-}$



## Heterolactic:

Phosphoketolase pathway

$ Glucose \rightarrow CO_2 + Ethanol + Pyruvate $

$ Pyruvate \rightarrow Lactate $

In [198]:
catab_donor = c_C6H12O6
catab_acceptor = c_C6H12O6

#Pyruvate -> Lactate
cat_A = balance_stoichiometry({"e-1","H+1", "C3H3O3-1"}, {"C3H5O3-1"})

#Glucose -> CO2 + Ethanol + Pyruvate

#cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"HCO3-1", "C2H6O", "C3H3O3-1", "e-1", "H+1"})
cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"C6H10O7-2", "e-1", "H+1"})
#cat_D = ({"C6H12O6":-1}, {"C3H5O3-1": 1, "C2H6O": 1, "HCO3-1": 1,  })

D_r = cat_D[0]
D_p = cat_D[1]
#Re-insert the correct ratio of products
D_p["HCO3-1"] = D_p["C6H10O7-2"]
D_p["C2H6O"] = D_p["C6H10O7-2"]
D_p["C3H3O3-1"] = D_p["C6H10O7-2"]
#Clear out the "fake" reactant
del D_p["C6H10O7-2"]

cat_D=(D_r,D_p)

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C6H12O6','NH4+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D

print anab
DGcatab_cor = calcDG(catab)
DGanab_cor = calcDG(anab)

#Corrections
DGcatab_cor += R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor += R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_C6H12O6")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n#### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
c_C2H6O + c_C3H5O3_n1 - c_C6H12O6 - c_H2O + c_HCO3_n1 + 2.0*c_H_p1

Anabolism:
-0.166666666666667*c_C6H12O6 + 0.5*c_H2O - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1
0.1*c_C2H6O + 0.1*c_C3H3O3_n1 - 0.266666666666667*c_C6H12O6 + 0.4*c_H2O + 0.1*c_HCO3_n1 + 0.4*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -208.666563
anab = -41.023313


catab = -208.666563
anab = -41.023313
dis = 236.050884
Yield = 1.069934



#### Yield = 1.0699
Catab

$1.0 H_{2}O+1.0 C_{6}H_{12}O_{6} \rightarrow 1.0.C_{2}H_{6}O+2.0.H^{+}+1.0.HCO_{3}^{-}+1.0.C_{3}H_{5}O_{3}^{-}$


Anab

$0.3 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+0.4.H^{+}+0.1.C_{2}H_{6}O+0.1.HCO_{3}^{-}+0.4.H_{2}O+0.1.C_{3}H_{3}O_{3}^{-}$




#### Yield = 1.0699
Catab

$1.0 H_{2}O+1.0 C_{6}H_{12}O_{6} \rightarrow 1.0.C_{2}H_{6}O+2.0.H^{+}+1.0.HCO_{3}^{-}+1.0.C_{3}H_{5}O_{3}^{-}$


Anab

$0.3 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+0.4.H^{+}+0.1.C_{2}H_{6}O+0.1.HCO_{3}^{-}+0.4.H_{2}O+0.1.C_{3}H_{3}O_{3}^{-}$

## Homolactic:

$ Glucose \rightarrow Lactate $

Via Pyruvate

In [199]:
catab_donor = c_C6H12O6
#catab_acceptor = c_C3H3O3_n1

cat_A = balance_stoichiometry({"e-1","H+1","C3H3O3-1"}, {"C3H5O3-1"})
#cat_D = balance_stoichiometry({"C6H12O6", "H2O"}, {"HCO3-1", "C2H3O2-1", "e-1", "H+1"})
cat_D = balance_stoichiometry({"C6H12O6"}, {"C3H3O3-1", "H+1", "e-1"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)

catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C6H12O6','NH4+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D

print anab
DGcatab_cor = calcDG(catab)
DGanab_cor = calcDG(anab)

#Corrections
DGcatab_cor += R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor += R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_C6H12O6")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
2.0*c_C3H5O3_n1 - c_C6H12O6 + 2.0*c_H_p1

Anabolism:
-0.166666666666667*c_C6H12O6 + 0.5*c_H2O - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1
0.1*c_C3H3O3_n1 - 0.216666666666667*c_C6H12O6 + 0.5*c_H2O + 0.3*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -194.266563
anab = -29.869984


catab = -194.266563
anab = -29.869984
dis = 236.050884
Yield = 0.942214



### Yield = 0.9422
Catab

$1.0 C_{6}H_{12}O_{6} \rightarrow 2.0.C_{3}H_{5}O_{3}^{-}+2.0.H^{+}$


Anab

$0.2 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 0.5.H_{2}O+0.1.C_{3}H_{3}O_{3}^{-}+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.3.H^{+}$




### Yield = 0.9422
Catab

$1.0 C_{6}H_{12}O_{6} \rightarrow 2.0.C_{3}H_{5}O_{3}^{-}+2.0.H^{+}$


Anab

$0.2 C_{6}H_{12}O_{6}+0.2 NH_{4}^{+} \rightarrow 0.5.H_{2}O+0.1.C_{3}H_{3}O_{3}^{-}+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.3.H^{+}$


# Amino acid fermentation : Stickland
    
$ Alanine + Glycine \rightarrow Acetate + NH_3 $

Don't have energetics for Alanine and Glycine - Alaninate etc

In [16]:
catab_donor = c_C3H7O2N

#Need to combine C fermentation products then split 

#cat_D = balance_stoichiometry({"C3H7O2", "H2O"}, {"H+1", "e-1", "HCO3-1", "C2H3O2-1"})
cat_D = balance_stoichiometry({"C3H7O2N", "H2O"}, {"H+1", "e-1", "C3H4O5-2", "NH4+1"})

#Split up the reac/prods
D_r = cat_D[0]
D_p = cat_D[1]
#Re-insert the correct ratio of products
D_p["HCO3-1"] = D_p["C3H4O5-2"]
D_p["C2H3O2-1"] = D_p["C3H4O5-2"]
#Clear out the "fake" reactant
del D_p["C3H4O5-2"]

cat_D=(D_r,D_p)


cat_A = balance_stoichiometry({"C2H5O2N", "e-1", "H+1"}, {"C2H3O2-1", "NH4+1"})


catab_D = reacToEquation(cat_D)

catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

#Assume Alanate is C and N source
anab_reac = balance_stoichiometry({'C3H7O2N', 'H+1', "NH4+1"}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_C3H7O2N")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

NameError: name 'balance_stoichiometry' is not defined

### Yield = 3.2891
Catab

$1.0 C_{3}H_{7}O_{2}N+3.0 H_{2}O+2.0 C_{2}H_{5}O_{2}N \rightarrow 3.0.C_{2}H_{3}O_{2}^{-}+3.0.NH_{4}^{+}+1.0.HCO_{3}^{-}+1.0.H^{+}$


Anab

$0.4 C_{3}H_{7}O_{2}N+0.1 H^{+} \rightarrow 0.0.H_{2}O+0.1.C_{2}H_{3}O_{2}^{-}+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.2.NH_{4}^{+}+0.1.HCO_{3}^{-}$

## Sulphur Oxidation:

$ HS^- \rightarrow S^0 + H^+ + e^-$

$ O_2 + e^- + H^+ \rightarrow H_2O $

In [202]:
catab_donor = c_HS_n1

cat_D = balance_stoichiometry({"HS-1"}, {"S", "H+1","e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
c_H2O - c_HS_n1 - c_H_p1 - 0.5*c_O2 + c_S

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
2.5*c_H2O - c_HCO3_n1 - 2.1*c_HS_n1 - 2.9*c_H_p1 - 0.2*c_NH4_p1 + 2.1*c_S + 1.0*c_X


catab = -209.366718
anab = 33.176516


catab = -209.366718
anab = 33.176516
dis = 1088.093713
Yield = 0.186723



### Yield = 0.1867
Catab

$1.0 HS^{-}+0.5 O_{2}+1.0 H^{+} \rightarrow 1.0.S+1.0.H_{2}O$


Anab

$0.2 NH_{4}^{+}+2.9 H^{+}+1.0 HCO_{3}^{-}+2.1 HS^{-} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+2.1.S$




### Yield = 0.1867
Catab

$1.0 HS^{-}+0.5 O_{2}+1.0 H^{+} \rightarrow 1.0.S+1.0.H_{2}O$


Anab

$0.2 NH_{4}^{+}+2.9 H^{+}+1.0 HCO_{3}^{-}+2.1 HS^{-} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+2.1.S$


In [441]:
print DGcatab_cor 
print DGanab_cor

-209.366718499399
33.1765163517414


### All the way to Sulphate

$ HS^- +H_2O \rightarrow SO_4^{2-} + H^+ + e^-$

$ O_2 + e^- + H^+ \rightarrow H_2O $

In [203]:
catab_donor = c_HS_n1

cat_D = balance_stoichiometry({"HS-1","H2O"}, {"SO4-2", "H+1","e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_HS_n1 + c_H_p1 - 2.0*c_O2 + c_SO4_n2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
0.4*c_H2O - c_HCO3_n1 - 0.525*c_HS_n1 - 0.274999999999999*c_H_p1 - 0.2*c_NH4_p1 + 0.525*c_SO4_n2 + 1.0*c_X


catab = -796.633282
anab = 54.614152


catab = -796.633282
anab = 54.614152
dis = 1088.093713
Yield = 0.697145



### Yield = 0.6971
Catab

$1.0 HS^{-}+2.0 O_{2} \rightarrow 1.0.H^{+}+1.0.SO_{4}^{2-}$


Anab

$0.3 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+0.5 HS^{-} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.5.SO_{4}^{2-}$




### Yield = 0.6971
Catab

$1.0 HS^{-}+2.0 O_{2} \rightarrow 1.0.H^{+}+1.0.SO_{4}^{2-}$


Anab

$0.3 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+0.5 HS^{-} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.5.SO_{4}^{2-}$


## Sulphur to Sulphate

$ S^0 \rightarrow SO_4 $

$ S^0 +H_2O \rightarrow SO_4^{2-} + H^+ + e^-$

$ O_2 + e^- + H^+ \rightarrow H_2O $

In [204]:
catab_donor = c_S

cat_D = balance_stoichiometry({"S","H2O"}, {"SO4-2", "H+1","e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_H2O + 2.0*c_H_p1 - 1.5*c_O2 - c_S + c_SO4_n2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-0.3*c_H2O - c_HCO3_n1 + 0.600000000000001*c_H_p1 - 0.2*c_NH4_p1 - 0.7*c_S + 0.7*c_SO4_n2 + 1.0*c_X


catab = -587.266563
anab = 61.760031


catab = -587.266563
anab = 61.760031
dis = 1088.093713
Yield = 0.510732



### Yield = 0.5107
Catab

$1.0 S+1.0 H_{2}O+1.5 O_{2} \rightarrow 2.0.H^{+}+1.0.SO_{4}^{2-}$


Anab

$0.3 H_{2}O+0.7 S+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+0.6.H^{+}+0.7.SO_{4}^{2-}$




### Yield = 0.5107
Catab

$1.0 S+1.0 H_{2}O+1.5 O_{2} \rightarrow 2.0.H^{+}+1.0.SO_{4}^{2-}$


Anab

$0.3 H_{2}O+0.7 S+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+0.6.H^{+}+0.7.SO_{4}^{2-}$


# AOB
 
$ NH_4^+ + H_2O \rightarrow NO_2^- +e^- + H^+  $

$ O_2 + e^- + H^+ \rightarrow H_2O $

Why ammonia vs ammonium? Rebeca uses Ammonia... and also generates $H_2$ which looks wrong

In [22]:
catab_donor = c_NH3

cat_D = balance_stoichiometry({"NH3","H2O"}, {"NO2-1", "H+1","e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
#Needs RET
DGdis = 3500
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
c_H2O + c_H_p1 - c_NH3 + c_NO2_n1 - 1.5*c_O2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
1.1*c_H2O - c_HCO3_n1 - 0.0999999999999996*c_H_p1 - 0.7*c_NH3 - 0.2*c_NH4_p1 + 0.7*c_NO2_n1 + 1.0*c_X


catab = -282.763282
anab = 274.912328


catab = -282.763282
anab = 274.912328
dis = 3500.000000
Yield = 0.074906



### Yield = 0.0749
Catab

$1.0 NH_{3}+1.5 O_{2} \rightarrow 1.00.H_{2}O+1.00.H^{+}+1.00.NO_{2}^{-}$


Anab

$0.1 H^{+}+0.7 NH_{3}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.10.H_{2}O+1.00 CH_{1.8}O_{0.5}N_{0.2}+0.70.NO_{2}^{-}$




### Yield = 0.1967
Catab

$1.5 O_{2}+1.0 NH_{4}^{+} \rightarrow 1.0.H_{2}O+2.0.H^{+}+1.0.NO_{2}^{-}$


Anab

$0.9 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+1.1.H_{2}O+0.6.H^{+}+0.7.NO_{2}^{-}$


In [64]:
catab_D

-2*c_H2O + 7*c_H_p1 - c_NH3 + c_NO2_n1 + 6*c_e_n1

In [65]:
catab_A

2*c_H2O - 4*c_H_p1 - c_O2 - 4*c_e_n1

# NOB

$ NO_2^- + H_2O \rightarrow NO_3^- + e^- $

In [23]:
catab_donor = c_NO2_n1

cat_D = balance_stoichiometry({"NO2-1","H2O"}, {"NO3-1", "H+1","e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NO2-1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")

#With RET
DGdis = 3500

lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_NO2_n1 + c_NO3_n1 - 0.5*c_O2

Anabolism:
2.9*c_H2O - c_HCO3_n1 - 6.6*c_H_p1 - 0.2*c_NO2_n1 + 1.0*c_X - 5.4*c_e_n1
0.2*c_H2O - c_HCO3_n1 - 1.2*c_H_p1 - 2.9*c_NO2_n1 + 2.7*c_NO3_n1 + 1.0*c_X


catab = -79.100000
anab = 313.249938


catab = -79.100000
anab = 313.249938
dis = 3500.000000
Yield = 0.020743



### Yield = 0.0207
Catab

$0.5 O_{2}+1.0 NO_{2}^{-} \rightarrow 1.00.NO_{3}^{-}$


Anab

$2.9 NO_{2}^{-}+1.2 H^{+}+1.0 HCO_{3}^{-} \rightarrow 1.00 CH_{1.8}O_{0.5}N_{0.2}+0.20.H_{2}O+2.70.NO_{3}^{-}$




### Yield = 0.0564
Catab

$0.5 O_{2}+1.0 NO_{2}^{-} \rightarrow 1.0.NO_{3}^{-}$


Anab

$2.9 NO_{2}^{-}+1.2 H^{+}+1.0 HCO_{3}^{-} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+0.2.H_{2}O+2.7.NO_{3}^{-}$


## ANAMOX

In [240]:
catab_D

4*c_H_p1 + c_N2 - c_N2H4 + 4*c_e_n1

In [239]:
catab_A

2*c_H2O - 4*c_H_p1 + c_N2H4 - c_NH4_p1 - c_NO2_n1 - 4*c_e_n1

In [24]:
catab_donor = c_NH4_p1

cat_D = balance_stoichiometry({"NH4+1"}, {"N2", "H+1","e-1"})
#Assuming 1:1 NO2- and NH4+:
cat_A = balance_stoichiometry({"NO2-1","H+1", "e-1"}, { "H2O", "N2"})

#A_r = cat_A[0]
#A_p = cat_A[1]
#Re-insert the correct ratio of products
#A_r["NH4+1"] = A_r["N2O2H4"]
#A_r["NO2-1"] = A_r["N2O2H4"]

#Clear out the "fake" reactant
#del A_r["N2O2H4"]

#cat_A=(A_r,A_p)

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/ \
    (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NO2-1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")

#Correction with RET
DGdis = 3500

lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
2.0*c_H2O + c_N2 - c_NH4_p1 - c_NO2_n1

Anabolism:
2.9*c_H2O - c_HCO3_n1 - 6.6*c_H_p1 - 0.2*c_NO2_n1 + 1.0*c_X - 5.4*c_e_n1
2.9*c_H2O - c_HCO3_n1 + 0.600000000000001*c_H_p1 + 0.9*c_N2 - 1.8*c_NH4_p1 - 0.2*c_NO2_n1 + 1.0*c_X


catab = -362.800000
anab = -42.579969


catab = -362.800000
anab = -42.579969
dis = 3500.000000
Yield = 0.104934



### Yield = 0.1049
Catab

$1.0 NH_{4}^{+}+1.0 NO_{2}^{-} \rightarrow 1.00.N_{2}+2.00.H_{2}O$


Anab

$0.2 NO_{2}^{-}+1.8 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.00 CH_{1.8}O_{0.5}N_{0.2}+2.90.H_{2}O+0.60.H^{+}+0.90.N_{2}$




### Yield = 0.3470
Catab

$1.0 NH_{4}^{+}+1.0 NO_{2}^{-} \rightarrow 1.0.N_{2}+2.0.H_{2}O$


Anab

$0.2 NO_{2}^{-}+1.8 NH_{4}^{+}+1.0 HCO_{3}^{-} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+2.9.H_{2}O+0.6.H^{+}+0.9.N_{2}$



# Methanogenesis

$ H_2 + CO_2 \rightarrow CH_4 + H_2O $

In [208]:
catab_donor = c_H2

cat_D = balance_stoichiometry({"H2"}, {"H+1", "e-1"})

cat_A = balance_stoichiometry({"HCO3-1", "e-1", "H+1"}, {"H2O", "CH4"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)



print "\n\n\n#### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
0.25*c_CH4 - c_H2 + 0.75*c_H2O - 0.25*c_HCO3_n1 - 0.25*c_H_p1

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-2.1*c_H2 + 2.5*c_H2O - c_HCO3_n1 - 0.8*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -33.866680
anab = -25.273375


catab = -33.866680
anab = -25.273375
dis = 1088.093713
Yield = 0.031865



#### Yield = 0.0319
Catab

$0.2 HCO_{3}^{-}+0.2 H^{+}+1.0 H_{2} \rightarrow 0.8.H_{2}O+0.2.CH_{4}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




#### Yield = 0.0319
Catab

$0.2 HCO_{3}^{-}+0.2 H^{+}+1.0 H_{2} \rightarrow 0.8.H_{2}O+0.2.CH_{4}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$


**Methylotrophic**

$ CH_3OH \rightarrow CH_4 + H_2O + CO_2 $

In [209]:
catab_donor = c_CH4O

cat_D = balance_stoichiometry({"CH4O", "H2O"}, {"H+1", "e-1", "HCO3-1"})

cat_A = balance_stoichiometry({"HCO3-1", "e-1", "H+1"}, {"H2O", "CH4"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'CH4O','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_CH4O")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
0.75*c_CH4 - c_CH4O + 0.25*c_H2O + 0.25*c_HCO3_n1 + 0.25*c_H_p1

Anabolism:
-c_CH4O + 0.5*c_H2O + 2.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X + 1.8*c_e_n1
0.225*c_CH4 - c_CH4O + 1.175*c_H2O - 0.225*c_HCO3_n1 - 0.0249999999999999*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -78.633320
anab = -32.786668


catab = -78.633320
anab = -32.786668
dis = 651.083763
Yield = 0.127177



### Yield = 0.1272
Catab

$1.0 CH_{4}O \rightarrow 0.2.HCO_{3}^{-}+0.2.H_{2}O+0.2.H^{+}+0.8.CH_{4}$


Anab

$0.0 H^{+}+0.2 NH_{4}^{+}+0.2 HCO_{3}^{-}+1.0 CH_{4}O \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+1.2.H_{2}O+0.2.CH_{4}$




### Yield = 0.1272
Catab

$1.0 CH_{4}O \rightarrow 0.2.HCO_{3}^{-}+0.2.H_{2}O+0.2.H^{+}+0.8.CH_{4}$


Anab

$0.0 H^{+}+0.2 NH_{4}^{+}+0.2 HCO_{3}^{-}+1.0 CH_{4}O \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+1.2.H_{2}O+0.2.CH_{4}$



**Acetoclastic**

$ CH_3COOH \rightarrow CH_4 + H2O + CO_2 $

In [210]:
catab_donor = c_C2H3O2_n1

cat_D = balance_stoichiometry({"C2H3O2-1", "H2O"}, {"H+1", "e-1", "HCO3-1"})

cat_A = balance_stoichiometry({"HCO3-1", "e-1", "H+1"}, {"H2O", "CH4"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C2H3O2-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
DGdis = calcGDis("c_C2H3O2_n1")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_C2H3O2_n1 + c_CH4 - c_H2O + c_HCO3_n1

Anabolism:
-0.5*c_C2H3O2_n1 + 0.5*c_H2O - 0.5*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1
-0.525*c_C2H3O2_n1 + 0.4*c_H2O + 0.05*c_HCO3_n1 - 0.275*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -31.000000
anab = 29.571652


catab = -31.000000
anab = 29.571652
dis = 432.122391
Yield = 0.067144



### Yield = 0.0671
Catab

$1.0 H_{2}O+1.0 C_{2}H_{3}O_{2}^{-} \rightarrow 1.0.HCO_{3}^{-}+1.0.CH_{4}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$




### Yield = 0.0671
Catab

$1.0 H_{2}O+1.0 C_{2}H_{3}O_{2}^{-} \rightarrow 1.0.HCO_{3}^{-}+1.0.CH_{4}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$


# Methanotroph

$ CH_4 + O_2 \rightarrow CO_2 + H_2O $

In [222]:
catab_donor = c_CH4

cat_D = balance_stoichiometry({"CH4", "H2O"}, {"H+1", "e-1", "HCO3-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CH4")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_CH4 + c_H2O + c_HCO3_n1 + c_H_p1 - 2.0*c_O2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-0.525*c_CH4 + 0.925*c_H2O - 0.475*c_HCO3_n1 - 0.274999999999999*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -813.333282
anab = 45.846652


catab = -813.333282
anab = 45.846652
dis = 986.432683
Yield = 0.787900



### Yield = 0.7879
Catab

$2.0 O_{2}+1.0 CH_{4} \rightarrow 1.0.H_{2}O+1.0.HCO_{3}^{-}+1.0.H^{+}$


Anab

$0.3 H^{+}+0.5 CH_{4}+0.2 NH_{4}^{+}+0.5 HCO_{3}^{-} \rightarrow 0.9.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




### Yield = 1.2591
Catab

$2.0 O_{2}+1.0 CH_{4} \rightarrow 1.0.H_{2}O+1.0.HCO_{3}^{-}+1.0.H^{+}$


Anab

$1.0 CH_{4}+0.2 NH_{4}^{+}+0.9 O_{2} \rightarrow 1.0 CH_{1.8}O_{0.5}N_{0.2}+1.4.H_{2}O+0.2.H^{+}$


# Anaerobic reactions

In [213]:
#NOT WORKING!!!

catab_donor = c_C2H6O

#cat_D = balance_stoichiometry({"C2H6O", "H2O"}, {"H+1", "e-1", "C2H3O2-1"})
cat_A = balance_stoichiometry({"e-1", "H+1"}, {"H2"})

cat_D = balance_stoichiometry({"C2H6O", "H2O"}, {"H+1", "e-1", "C2H3O2-1"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C2H6O','NH4+1'}, {'C10H18O5N2', 'H+1','e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_C2H6O")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getHTMLReacts(anab,catab)

Catabolism:
c_C2H3O2_n1 - c_C2H6O + 2.0*c_H2 - c_H2O + c_H_p1

Anabolism:
-0.5*c_C2H6O + 2.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X + 1.8*c_e_n1
-0.5*c_C2H6O + 0.9*c_H2 + 0.2*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = 9.666718
anab = 31.793344


catab = 9.666718
anab = 31.793344
dis = 620.725668
Yield = -0.014814



### Yield = -0.0148
#### Catab
1.0 H<sub>2</sub>O+1.0 C<sub>2</sub>H<sub>6</sub>O &rarr; 1.0.H<sup>+</sup>+1.0.C<sub>2</sub>H<sub>3</sub>O<sub>2</sub><sup>-</sup>+2.0.H<sub>2</sub>
#### Anab
0.2 NH<sub>4</sub><sup>+</sup>+0.5 C<sub>2</sub>H<sub>6</sub>O &rarr; 0.2.H<sup>+</sup>+1.0 CH<sub>1.8</sub>O<sub>0.5</sub>N<sub>0.2</sub>+0.9.H<sub>2</sub>


In [438]:
print DGcatab_cor 
print DGanab_cor

9.66671849939954
31.7933436998799


# Sulfate Reducers

$ SO_4^{2-} + H_2 \rightarrow HS^- + H_2O $

In [214]:
catab_donor = c_H2

cat_D = balance_stoichiometry({"H2" }, {"H+1", "e-1"})

cat_A = balance_stoichiometry({"SO4-2", "e-1", "H+1"}, {"HS-1","H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_H2 + c_H2O + 0.25*c_HS_n1 - 0.25*c_H_p1 - 0.25*c_SO4_n2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-2.1*c_H2 + 2.5*c_H2O - c_HCO3_n1 - 0.8*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -38.041680
anab = -25.273375


catab = -38.041680
anab = -25.273375
dis = 1088.093713
Yield = 0.035793



### Yield = 0.0358
Catab

$0.2 H^{+}+0.2 SO_{4}^{2-}+1.0 H_{2} \rightarrow 1.0.H_{2}O+0.2.HS^{-}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




### Yield = 0.0358
Catab

$0.2 H^{+}+0.2 SO_{4}^{2-}+1.0 H_{2} \rightarrow 1.0.H_{2}O+0.2.HS^{-}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$



In [34]:
print DGcatab_cor 
print DGanab_cor

-38.0416796248499
-25.2733747995197


## Sulfate with propionate to acetate

$ CH_3CH_2COO^{-1} + SO_4^{2-} \rightarrow CH3COO^- + HS^- $

In [215]:
catab_donor = c_C3H5O2_n1

cat_D = balance_stoichiometry({"C3H5O2-1", "H2O" }, {"H+1",  "e-1", "C2H3O2-1"})

cat_A = balance_stoichiometry({"SO4-2", "e-1", "H+1"}, {"HS-1","H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C3H5O2-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_C3H5O2_n1")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)

print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
1.5*c_C2H3O2_n1 - c_C3H5O2_n1 + 0.25*c_HS_n1 + 0.25*c_H_p1 - 0.25*c_SO4_n2

Anabolism:
-0.333333333333333*c_C3H5O2_n1 + 0.166666666666667*c_H2O + 0.333333333333333*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X + 0.466666666666667*c_e_n1
-0.333333333333333*c_C3H5O2_n1 + 0.4*c_H2O + 0.0583333333333333*c_HS_n1 - 0.191666666666667*c_H_p1 - 0.2*c_NH4_p1 - 0.0583333333333333*c_SO4_n2 + 1.0*c_X


catab = -13.808320
anab = 26.161379


catab = -13.808320
anab = 26.161379
dis = 376.809149
Yield = 0.034266



### Yield = 0.0343
Catab

$0.2 SO_{4}^{2-}+1.0 C_{3}H_{5}O_{2}^{-} \rightarrow 0.2.HS^{-}+0.2.H^{+}+1.5.C_{2}H_{3}O_{2}^{-}$


Anab

$0.2 H^{+}+0.3 C_{3}H_{5}O_{2}^{-}+0.2 NH_{4}^{+}+0.1 SO_{4}^{2-} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HS^{-}$




### Yield = 0.0343
Catab

$0.2 SO_{4}^{2-}+1.0 C_{3}H_{5}O_{2}^{-} \rightarrow 0.2.HS^{-}+0.2.H^{+}+1.5.C_{2}H_{3}O_{2}^{-}$


Anab

$0.2 H^{+}+0.3 C_{3}H_{5}O_{2}^{-}+0.2 NH_{4}^{+}+0.1 SO_{4}^{2-} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HS^{-}$

$ CH_3COO^- + SO_4^{2-} \rightarrow CO_2 + H_2O + H_2S $

In [216]:
catab_donor = c_C2H3O2_n1

cat_D = balance_stoichiometry({"C2H3O2-1", "H2O" }, {"H+1",  "e-1", "HCO3-1"})

cat_A = balance_stoichiometry({"SO4-2", "e-1", "H+1"}, {"HS-1","H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'C2H3O2-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_C2H3O2_n1")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_C2H3O2_n1 + 2.0*c_HCO3_n1 + c_HS_n1 - c_SO4_n2

Anabolism:
-0.5*c_C2H3O2_n1 + 0.5*c_H2O - 0.5*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 0.2*c_e_n1
-0.525*c_C2H3O2_n1 + 0.4*c_H2O + 0.05*c_HCO3_n1 - 0.275*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -47.700000
anab = 29.571652


catab = -47.700000
anab = 29.571652
dis = 432.122391
Yield = 0.103315



### Yield = 0.1033
Catab

$1.0 C_{2}H_{3}O_{2}^{-}+1.0 SO_{4}^{2-} \rightarrow 2.0.HCO_{3}^{-}+1.0.HS^{-}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$




### Yield = 0.1033
Catab

$1.0 C_{2}H_{3}O_{2}^{-}+1.0 SO_{4}^{2-} \rightarrow 2.0.HCO_{3}^{-}+1.0.HS^{-}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$



## SRB C sources and e<sup>-</sup> donors

Still to do:

 - Lactate
 - Pyruvate
 - Ethanol
 - Fumarate
 - Malate
 - Choline
 - Acetate
 - Propionate
 - Butyrate
 - Benzoate
 - Indole



$ S^0 + H_2 \rightarrow H_2S $

In [217]:
catab_donor = c_H2

cat_D = balance_stoichiometry({"H2"}, {"H+1", "e-1"})

cat_A = balance_stoichiometry({"S", "e-1", "H+1"}, {"HS-1"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)



print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_H2 + c_HS_n1 + c_H_p1 - c_S

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-2.1*c_H2 + 2.5*c_H2O - c_HCO3_n1 - 0.8*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -27.833282
anab = -25.273375


catab = -27.833282
anab = -25.273375
dis = 1088.093713
Yield = 0.026188



### Yield = 0.0262
Catab

$1.0 S+1.0 H_{2} \rightarrow 1.0.HS^{-}+1.0.H^{+}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




### Yield = 0.0262
Catab

$1.0 S+1.0 H_{2} \rightarrow 1.0.HS^{-}+1.0.H^{+}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$

# Acetogenesis

$ H_2 + H^+ + HCO_3^- \rightarrow CH_3COO^- + H_2O $

In [218]:
catab_donor = c_H2

cat_D = balance_stoichiometry({"H2"}, {"H+1", "e-1"})

cat_A = balance_stoichiometry({"HCO3-1", "e-1", "H+1"}, {"C2H3O2-1", "H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
0.25*c_C2H3O2_n1 - c_H2 + c_H2O - 0.5*c_HCO3_n1 - 0.25*c_H_p1

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-2.1*c_H2 + 2.5*c_H2O - c_HCO3_n1 - 0.8*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -26.116680
anab = -25.273375


catab = -26.116680
anab = -25.273375
dis = 1088.093713
Yield = 0.024573



### Yield = 0.0246
Catab

$0.2 H^{+}+0.5 HCO_{3}^{-}+1.0 H_{2} \rightarrow 1.0.H_{2}O+0.2.C_{2}H_{3}O_{2}^{-}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




### Yield = 0.0246
Catab

$0.2 H^{+}+0.5 HCO_{3}^{-}+1.0 H_{2} \rightarrow 1.0.H_{2}O+0.2.C_{2}H_{3}O_{2}^{-}$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$



# H<sub>2</sub> oxidation

$H_2 + O_2 \rightarrow H_2O $

In [219]:
catab_donor = c_H2

cat_D = balance_stoichiometry({"H2"}, {"H+1", "e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))
print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_H2 + c_H2O - 0.5*c_O2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-2.1*c_H2 + 2.5*c_H2O - c_HCO3_n1 - 0.8*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -237.200000
anab = -25.273375


catab = -237.200000
anab = -25.273375
dis = 1088.093713
Yield = 0.223180



### Yield = 0.2232
Catab

$0.5 O_{2}+1.0 H_{2} \rightarrow 1.0.H_{2}O$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$




### Yield = 0.2232
Catab

$0.5 O_{2}+1.0 H_{2} \rightarrow 1.0.H_{2}O$


Anab

$0.8 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+2.1 H_{2} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}$


# Anaerobic Fe reduction:

Geobacter, Geospirillum, Geovibrio

$ CH_3COO^- + Fe^{3+} \rightarrow CO_2 + Fe^{2+} $

In [220]:
catab_donor = c_C2H3O2_n1

cat_D = balance_stoichiometry({"C2H3O2-1", "H2O"}, {"H+1", "e-1", "HCO3-1"})

cat_A = balance_stoichiometry({"Fe+3", "e-1"}, {"Fe+2"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_C2H3O2_n1 + 8.0*c_Fe_p2 - 8.0*c_Fe_p3 - 4.0*c_H2O + 2.0*c_HCO3_n1 + 9.0*c_H_p1

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-0.525*c_C2H3O2_n1 + 0.4*c_H2O + 0.05*c_HCO3_n1 - 0.274999999999999*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -809.399534
anab = 29.571652


catab = -809.399534
anab = 29.571652
dis = 1088.093713
Yield = 0.724188



### Yield = 0.7242
Catab

$1.0 C_{2}H_{3}O_{2}^{-}+4.0 H_{2}O+8.0 Fe^{3+} \rightarrow 9.0.H^{+}+2.0.HCO_{3}^{-}+8.0.Fe^{2+}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$




### Yield = 0.7242
Catab

$1.0 C_{2}H_{3}O_{2}^{-}+4.0 H_{2}O+8.0 Fe^{3+} \rightarrow 9.0.H^{+}+2.0.HCO_{3}^{-}+8.0.Fe^{2+}$


Anab

$0.5 C_{2}H_{3}O_{2}^{-}+0.3 H^{+}+0.2 NH_{4}^{+} \rightarrow 0.4.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+0.1.HCO_{3}^{-}$



# Fe<sup>II</sup> Oxidation

## Aerobic

In [247]:
catab_donor = c_Fe_p2

cat_D = balance_stoichiometry({"Fe+2"}, {"Fe+3", "e-1"})

cat_A = balance_stoichiometry({"O2", "e-1", "H+1"}, {"H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_Fe_p2 + c_Fe_p3 + 0.5*c_H2O - c_H_p1 - 0.25*c_O2

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-4.2*c_Fe_p2 + 4.2*c_Fe_p3 + 2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = -4.366718
anab = 454.506408


catab = -4.366718
anab = 454.506408
dis = 1088.093713
Yield = 0.002831



### Yield = 0.0028
Catab

$1.0 H^{+}+0.2 O_{2}+1.0 Fe^{2+} \rightarrow 0.5.H_{2}O+1.0.Fe^{3+}$


Anab

$5.0 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+4.2 Fe^{2+} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+4.2.Fe^{3+}$




### Yield = 0.0028
Catab

$1.0 H^{+}+0.2 O_{2}+1.0 Fe^{2+} \rightarrow 0.5.H_{2}O+1.0.Fe^{3+}$


Anab

$5.0 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+4.2 Fe^{2+} \rightarrow 2.5.H_{2}O+1.0 CH_{1.8}O_{0.5}N_{0.2}+4.2.Fe^{3+}$


## Anaerobic (1) 

$NO_2^-$

In [254]:
catab_donor = c_Fe_p2

cat_D = balance_stoichiometry({"Fe+2"}, {"Fe+3", "e-1"})

cat_A = balance_stoichiometry({"NO3-1", "e-1", "H+1"}, {"N2","H2O"})

catab_D = reacToEquation(cat_D)
catab_A = reacToEquation(cat_A)
catab = (catab_A.coeff(c_e_n1) * catab_D - catab_D.coeff(c_e_n1)*catab_A)/        (catab_D.coeff(c_e_n1)*catab_A.coeff(catab_donor) - catab_A.coeff(c_e_n1)*catab_D.coeff(catab_donor))

anab_reac = balance_stoichiometry({'HCO3-1','NH4+1', 'H+1'}, {'C10H18O5N2', 'H2O', 'e-1'})
anab = reacToEquation(anab_reac)
anab = anab/(anab.coeff(c_X)*10) + 0.9*c_X #To bring c_X back up to 1

catab = sympy.N(catab)
anab = sympy.N(anab)

print "Catabolism:"
print catab

print
print "Anabolism:"
print anab


if anab.coeff(c_e_n1) > 0.01:
    #Generates electrons - absorb with the acceptor:
    anab = anab + abs(anab.coeff(c_e_n1)/catab_A.coeff(c_e_n1))*catab_A
elif anab.coeff(c_e_n1) <0.01:
    #Needs extra electrons from e-donor
    anab = anab  + abs(anab.coeff(c_e_n1)/catab_D.coeff(c_e_n1))*catab_D
    
print anab
    
DGcatab = calcDG(catab)
DGanab = calcDG(anab)

#Corrections
DGcatab_cor = DGcatab + R*T*log(0.0000001**(float(catab.coeff(c_H_p1))))
DGanab_cor = DGanab + R*T*log(0.0000001**(float(anab.coeff(c_H_p1))))

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor

DGdis = calcGDis("c_CO2")
lambdacat = (DGanab_cor + DGdis)/(-1.0*DGcatab_cor)

print "\n\ncatab = %f" % DGcatab_cor
print "anab = %f" % DGanab_cor
print "dis = %f" % DGdis
print "Yield = %f" % (1.0/lambdacat)


print "\n\n\n### Yield = %0.4f" %  (1.0/lambdacat) 
getLaTeXReacts(anab,catab)

Catabolism:
-c_Fe_p2 + c_Fe_p3 + 0.6*c_H2O - 1.2*c_H_p1 + 0.1*c_N2 - 0.2*c_NO3_n1

Anabolism:
2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X - 4.2*c_e_n1
-4.2*c_Fe_p2 + 4.2*c_Fe_p3 + 2.5*c_H2O - c_HCO3_n1 - 5.0*c_H_p1 - 0.2*c_NH4_p1 + 1.0*c_X


catab = 2.159938
anab = 454.506408


catab = 2.159938
anab = 454.506408
dis = 1088.093713
Yield = -0.001400



### Yield = -0.0014
Catab

$1.2 H^{+}+1.0 Fe^{2+}+0.2 NO_{3}^{-} \rightarrow 0.60.H_{2}O+1.00.Fe^{3+}+0.10.N_{2}$


Anab

$5.0 H^{+}+0.2 NH_{4}^{+}+1.0 HCO_{3}^{-}+4.2 Fe^{2+} \rightarrow 2.50.H_{2}O+1.00 CH_{1.8}O_{0.5}N_{0.2}+4.20.Fe^{3+}$


